# Computer Vision Analysis
#### By Ronny Toribio, Kadir O. Altunel, Michael Cook-Stahl
#### Based on [Hands on Machine Learning 2nd edition](https://github.com/ageron/handson-ml2/), [FER2013 candidate 1](https://www.kaggle.com/code/ritikjain00/model-training-fer-13) and [FER2013 candidate 2](https://www.kaggle.com/code/gauravsharma99/facial-emotion-recognition/notebook)

### Import modules and declare constants

In [ ]:
%matplotlib inline
import os.path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

BATCH_SIZE = 32
IMAGE_ZOOM = 0.3
IMAGE_SHAPE = (48, 48)
INPUT_SHAPE = (48, 48, 1)

### Load Facial Emotion Recognition dataset
#### training, validation, and testing

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=IMAGE_ZOOM, horizontal_flip=True,
                                   validation_split=0.10)
Xy_train = train_datagen.flow_from_directory(os.path.join("fer2013", "train"), batch_size=BATCH_SIZE, 
                                   target_size=IMAGE_SHAPE, shuffle=True, subset="training",
                                   color_mode="grayscale", class_mode="categorical")

Xy_valid = train_datagen.flow_from_directory(os.path.join("fer2013", "train"), batch_size=BATCH_SIZE, 
                                   target_size=IMAGE_SHAPE, shuffle=True, subset="validation",
                                   color_mode="grayscale", class_mode="categorical")

test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)
Xy_test = test_datagen.flow_from_directory(os.path.join("fer2013", "test"), batch_size=BATCH_SIZE,
                                   target_size=IMAGE_SHAPE, shuffle=True,
                                   color_mode="grayscale", class_mode="categorical")

### Build CNN model

In [ ]:
model = Sequential([
    # Convolution Block 1
    Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu", kernel_initializer="he_normal", input_shape=INPUT_SHAPE),
    BatchNormalization(),
    Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu", kernel_initializer="he_normal"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"),
    Dropout(0.25),
    
    # Convolution Block 2
    Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu", kernel_initializer="he_normal", kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Conv2D(filters=256, kernel_size=(3, 3), padding="valid", activation="relu", kernel_initializer="he_normal", kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), padding="valid"),
    Dropout(0.25),
    
    # Classification Block
    Flatten(),
    Dense(1024, activation="relu", kernel_initializer="he_normal"),
    BatchNormalization(),
    Dropout(0.5),
    Dense(7, activation="softmax")
])

### Model properties

In [ ]:
model.layers

In [ ]:
model.summary()

### Compile model

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=Nadam(learning_rate=0.001, beta_1=0.9, epsilon=1e-07),
              metrics=["accuracy"])

### Save Architecture as JSON

In [ ]:
model_json = model.to_json()
with open("cnn_model.json") as f:
    f.write(model_json)

### Checkpoint callback

In [ ]:
checkpoint_cb = ModelCheckpoint("cnn_model_weights.h5", monitor="val_accuracy",
                                verbose=1, save_weights_only=True)

### Early stopping callback

In [ ]:
early_stopping_cb = EarlyStopping(monitor="val_accuracy", min_delta=0.00005,
                                  patience=11, verbose=1, restore_best_weights=True)

### Reduce learning rate on plateau callback

In [ ]:
reduce_lr_cb = ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=7, min_l=1e-7, verbose=1)

### CSV callback

In [ ]:
csv_cb = CSVLogger("cnn_model_training.txt")

### Train the model

In [ ]:
history = model.fit(Xy_train, epochs=60, validation_data=(Xy_valid),
                    steps_per_epoch=Xy_train.n // BATCH_SIZE,
                    validation_steps=Xy_valid.n // BATCH_SIZE,
                    callbacks=[checkpoint_cb, early_stopping_cb, reduce_lr_cb, csv_cb])

### Roll back to the best weights

In [ ]:
model.load_weights("cnn_model_weights.h5")

### Plot training history

In [ ]:
#adjusted_epoch = [x - 0.5 for x in history.epoch]
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
ax.grid(True)
ax.plot(history.epoch, history.history["loss"], "b-", label="loss")
ax.plot(history.epoch, history.history["accuracy"], "r-", label="accuracy")
ax.plot(history.epoch, history.history["val_loss"], "c-", label="val_loss")
ax.plot(history.epoch, history.history["val_accuracy"], "g-", label="val_accuracy")
ax.legend()
fig.savefig("cnn_model_training.png")
plt.show()

### Evaluate model

In [ ]:
print("Evaluate train set")
model.evaluate(Xy_train)
print("Evaluate validation set")
model.evaluate(Xy_valid)
print("Evaluate test set")
model.evaluate(Xy_test)